## Etapas
- 1. Explicação do Problema
- 2. Importar Bibliotecas e Base de Dados
- 3. Consolidar Base de Dados
- 4. Identificar colunas que são desnecessárias
- 5. Tratar valores faltantes
- 6. Verificar dtypes das colunas
- 7. Análise exploratória e tratar outliers
    - Dados Numéricos Contínuos e Discretos
    - Dados de Texto (categoria)
    - Dados em Lista
    - Dados em Mapa
- 8. Encoding
    - Dados True ou False
    - Dados Categoria Texto
- 9. Modelo de Previsão
    - Métricas de Avaliação
    - Definição do Tipo de Modelo
    - Escolha dos Modelos a Testar
    - Treino e Avaliação
- 10. Análise do Melhor Modelo

# Etapa 1

### Contexto
Temos uma base de dados de airbnbs no Rio de Janeiro, com diversas informações sobre os imóveis, separadas por mês. (A data é um fator importante pois a sazonalidade afeta o preço do aluguel)

### Objetivo
Definir quais variáveis podem ser utilizadas e criar um modelo para prever o preço ideal de um Airbnb com base em diferentes informações do imóvel.
Serve para donos de imóveis que querem entender o preço ideal para sua locação, ou para hóspedes que querem entender se o preço é justo ou não de um imóvel.


# Etapas 2 e 3

In [12]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
import os
import re

pd.options.display.max_rows = 4000
pd.options.display.max_seq_items = 200000
cwd = os.getcwd()

path_dataset = os.path.join(cwd, 'dataset')

dict_meses = {'jan': 1, 'fev': 2, 'mar': 3, 'abr': 4, 'mai': 5, 'jun': 6, 'jul': 7, 'ago': 8, 'set': 9, 'out': 10, 'nov': 11, 'dez': 12}


arquivos = os.listdir(path_dataset)

base_airbnb = pd.DataFrame()


for arquivo in arquivos:

    caminho_arquivo = os.path.join(path_dataset, arquivo)

    padrao_ano = re.compile('\d+')

    mes_texto = arquivo[:3]

    mes_int = dict_meses[mes_texto]


    ano = int(re.search(padrao_ano, arquivo).group(0))

    df_mes = pd.read_csv(caminho_arquivo, encoding= 'latin1')

    df_mes['month'] = mes_int

    df_mes['year'] = ano

    base_airbnb = pd.concat([base_airbnb, df_mes])



display(base_airbnb)


<>:26: SyntaxWarning: invalid escape sequence '\d'
<>:26: SyntaxWarning: invalid escape sequence '\d'
C:\Users\Vinicius\AppData\Local\Temp\ipykernel_19200\1882518515.py:26: SyntaxWarning: invalid escape sequence '\d'
  padrao_ano = re.compile('\d+')
C:\Users\Vinicius\AppData\Local\Temp\ipykernel_19200\1882518515.py:35: DtypeWarning: Columns (62,87) have mixed types. Specify dtype option on import or set low_memory=False.
  df_mes = pd.read_csv(caminho_arquivo, encoding= 'latin1')
C:\Users\Vinicius\AppData\Local\Temp\ipykernel_19200\1882518515.py:35: DtypeWarning: Columns (61,62,94) have mixed types. Specify dtype option on import or set low_memory=False.
  df_mes = pd.read_csv(caminho_arquivo, encoding= 'latin1')
C:\Users\Vinicius\AppData\Local\Temp\ipykernel_19200\1882518515.py:35: DtypeWarning: Columns (61,62,94) have mixed types. Specify dtype option on import or set low_memory=False.
  df_mes = pd.read_csv(caminho_arquivo, encoding= 'latin1')
C:\Users\Vinicius\AppData\Local\Temp\ip

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,...,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,number_of_reviews_ltm,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms
0,14063,https://www.airbnb.com/rooms/14063,20180414160018,2018-04-14,Living in a Postcard,"Besides the most iconic's view, our apartment ...",NaN,"Besides the most iconic's view, our apartment ...",none,Best and favorite neighborhood of Rio. Perfect...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,17878,https://www.airbnb.com/rooms/17878,20180414160018,2018-04-14,Very Nice 2Br - Copacabana - WiFi,Please note that special rates apply for New Y...,- large balcony which looks out on pedestrian ...,Please note that special rates apply for New Y...,none,This is the best spot in Rio. Everything happe...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,24480,https://www.airbnb.com/rooms/24480,20180414160018,2018-04-14,Nice and cozy near Ipanema Beach,My studio is located in the best of Ipanema. ...,The studio is located at Vinicius de Moraes St...,My studio is located in the best of Ipanema. ...,none,"The beach, the lagoon, Ipanema is a great loca...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,25026,https://www.airbnb.com/rooms/25026,20180414160018,2018-04-14,Beautiful Modern Decorated Studio in Copa,"Our apartment is a little gem, everyone loves ...",This newly renovated studio (last renovations ...,"Our apartment is a little gem, everyone loves ...",none,Copacabana is a lively neighborhood and the ap...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,31560,https://www.airbnb.com/rooms/31560,20180414160018,2018-04-14,NICE & COZY 1BDR - IPANEMA BEACH,This nice and clean 1 bedroom apartment is loc...,This nice and clean 1 bedroom apartment is loc...,This nice and clean 1 bedroom apartment is loc...,none,Die Nachbarschaft von Ipanema ist super lebend...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34324,38844730,https://www.airbnb.com/rooms/38844730,20190923212307,2019-09-24,TRANSAMERICA BARRA DA TIJUCA R IV,"Em estilo contemporÃ¢neo, o Transamerica Prime...",NaN,"Em estilo contemporÃ¢neo, o Transamerica Prime...",none,NaN,...,1.0,1.0,1125.0,1125.0,1.0,1125.0,0.0,15.0,0.0,0.0
34325,38846408,https://www.airbnb.com/rooms/38846408,20190923212307,2019-09-24,Alugo para o Rock in Rio,"ConfortÃ¡vel apartamento, 2 quartos , sendo 1 ...",O apartamento estarÃ¡ com mobÃ­lia completa di...,"ConfortÃ¡vel apartamento, 2 quartos , sendo 1 ...",none,"Muito prÃ³ximo ao Parque OlÃ­mpico, local do e...",...,2.0,2.0,1125.0,1125.0,2.0,1125.0,0.0,1.0,0.0,0.0
34326,38846703,https://www.airbnb.com/rooms/38846703,20190923212307,2019-09-24,Apt COMPLETO em COPACABANA c/TOTAL SEGURANÃA,Apartamento quarto e sala COMPLETO para curtas...,EspaÃ§o ideal para atÃ© 5 pessoas. Cama de cas...,Apartamento quarto e sala COMPLETO para curtas...,none,NaN,...,3.0,3.0,1125.0,1125.0,3.0,1125.0,0.0,23.0,6.0,0.0
34327,38847050,https://www.airbnb.com/rooms/38847050,20190923212307,2019-09-24,Cobertura Cinematografica,Cobertura alto nivel,NaN,Cobertura alto nivel,none,NaN,...,1.0,1.0,1125.0,1125.0,1.0,1125.0,0.0,1.0,0.0,0.0


- Número excessivo de colunas, pode tornar o modelo lento. Será feita uma análise rápida para identificar quais colunas podem ser excluídas da base
- Tipos de colunas que vamos excluir:
    - 1. IDs, links e informações não relevantes pro modelo.
    - 2. Colunas repetidas ou que entregam a mesma informação pro modelo. Ex: Data x Ano/Mês
    - 3. Colunas preenchidas com texto livre -> Não será rodada nenhuma análise de palavras
    - 4. Colunas com maioria ou todos os dados iguais
- Será criado um arquivo excel com os primeiros 1000 registros para fazer uma análise qualitativa

In [17]:
base_airbnb.head(1000).to_excel('Top1000.xlsx')

In [18]:
print(base_airbnb[['experiences_offered']].value_counts())

experiences_offered
none                   902210
Name: count, dtype: int64


In [28]:
print((base_airbnb['host_listings_count'] == base_airbnb['host_total_listings_count']).value_counts())

True     901750
False       460
Name: count, dtype: int64


In [30]:
print(base_airbnb['square_feet'].isnull().sum())
print(len(base_airbnb))

888162
902210


# Etapa 4
### Após a análise qualitativa pelo Excel, foram definidas as colunas que em tese impactam mais o preço de acordo com os critérios definidos acima

In [35]:
df_top1000 = pd.read_excel('Top1000.xlsx')

colunas_desejadas = df_top1000.columns
print(colunas_desejadas)
print(len(colunas_desejadas))

Index(['host_response_time', 'host_response_rate', 'host_is_superhost',
       'host_listings_count', 'latitude', 'longitude', 'property_type',
       'room_type', 'accommodates', 'bathrooms', 'bedrooms', 'beds',
       'bed_type', 'amenities', 'price', 'security_deposit', 'cleaning_fee',
       'guests_included', 'extra_people', 'minimum_nights', 'maximum_nights',
       'number_of_reviews', 'review_scores_rating', 'review_scores_accuracy',
       'review_scores_cleanliness', 'review_scores_checkin',
       'review_scores_communication', 'review_scores_location',
       'review_scores_value', 'instant_bookable', 'is_business_travel_ready',
       'cancellation_policy', 'month', 'year'],
      dtype='object')
34


In [33]:
df_airbnb = base_airbnb[colunas_desejadas]

display(df_airbnb)

,host_response_time,host_response_rate,host_is_superhost,host_listings_count,latitude,longitude,property_type,room_type,accommodates,bathrooms,...,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,instant_bookable,is_business_travel_ready,cancellation_policy,month,year
0,NaN,NaN,f,1.0,-22.946854,-43.182737,Apartment,Entire home/apt,4,1.0,...,9.0,9.0,9.0,9.0,9.0,f,f,strict_14_with_grace_period,4,2018
1,within an hour,100%,t,2.0,-22.965919,-43.178962,Condominium,Entire home/apt,5,1.0,...,9.0,10.0,10.0,9.0,9.0,t,f,strict,4,2018
2,within an hour,100%,f,1.0,-22.985698,-43.201935,Apartment,Entire home/apt,2,1.0,...,10.0,10.0,10.0,10.0,9.0,f,f,strict,4,2018
3,within an hour,100%,f,3.0,-22.977117,-43.190454,Apartment,Entire home/apt,3,1.0,...,10.0,10.0,10.0,10.0,9.0,f,f,strict,4,2018
4,within an hour,100%,t,1.0,-22.983024,-43.214270,Apartment,Entire home/apt,3,1.0,...,10.0,10.0,10.0,10.0,9.0,t,f,strict,4,2018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34324,within an hour,93%,f,0.0,-23.003180,-43.342840,Apartment,Entire home/apt,4,1.0,...,NaN,NaN,NaN,NaN,NaN,f,f,flexible,9,2019
34325,NaN,NaN,f,0.0,-22.966640,-43.393450,Apartment,Entire home/apt,4,2.0,...,NaN,NaN,NaN,NaN,NaN,f,f,flexible,9,2019
34326,within a few hours,74%,f,32.0,-22.962080,-43.175520,Apartment,Entire home/apt,5,1.0,...,NaN,NaN,NaN,NaN,NaN,f,f,strict_14_with_grace_period,9,2019
34327,NaN,NaN,f,0.0,-23.003400,-43.341820,Apartment,Entire home/apt,4,1.0,...,NaN,NaN,NaN,NaN,NaN,f,f,strict_14_with_grace_period,9,2019


# Etapa 5

### Tratar valores faltando

In [36]:
print(df_airbnb.isnull().sum())

host_response_time             401843
host_response_rate             401846
host_is_superhost                 460
host_listings_count               460
latitude                            0
longitude                           0
property_type                       0
room_type                           0
accommodates                        0
bathrooms                        1724
bedrooms                          850
beds                             2502
bed_type                            0
amenities                           0
price                               0
security_deposit               421280
cleaning_fee                   313506
guests_included                     0
extra_people                        0
minimum_nights                      0
maximum_nights                      0
number_of_reviews                   0
review_scores_rating           448016
review_scores_accuracy         448586
review_scores_cleanliness      448413
review_scores_checkin          448650
review_score

- Foram percebidas grandes quantidades de dados faltantes em certas colunas, iremos excluir as colunas com + de 300 mil valores NaN

In [37]:
for col in df_airbnb:

    if df_airbnb[col].isnull().sum() > 300000:
        df_airbnb = df_airbnb.drop(col, axis=1)

print(df_airbnb.isnull().sum())

host_is_superhost            460
host_listings_count          460
latitude                       0
longitude                      0
property_type                  0
room_type                      0
accommodates                   0
bathrooms                   1724
bedrooms                     850
beds                        2502
bed_type                       0
amenities                      0
price                          0
guests_included                0
extra_people                   0
minimum_nights                 0
maximum_nights                 0
number_of_reviews              0
instant_bookable               0
is_business_travel_ready       0
cancellation_policy            0
month                          0
year                           0
dtype: int64


- Agora, com poucos dados faltando, iremos excluir as poucas linhas que possuem valores nulos

In [40]:
df_airbnb = df_airbnb.dropna()

df_airbnb.shape

print(df_airbnb.isnull().sum())

host_is_superhost           0
host_listings_count         0
latitude                    0
longitude                   0
property_type               0
room_type                   0
accommodates                0
bathrooms                   0
bedrooms                    0
beds                        0
bed_type                    0
amenities                   0
price                       0
guests_included             0
extra_people                0
minimum_nights              0
maximum_nights              0
number_of_reviews           0
instant_bookable            0
is_business_travel_ready    0
cancellation_policy         0
month                       0
year                        0
dtype: int64
